In [14]:
# Importamos las librerias necesarias

import yfinance as yf
import pandas as pd
import numpy as np
import warnings 

# ETL

## Dataset 1 Compañias S&P 500


Se descarga la información de las compañias que estan en el S&P 500

In [11]:
data_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# Se selecciona el primer dataset de la url con la funcion read_html

SPCompanies = pd.read_html(data_url)[0]

In [4]:
SPCompanies

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [12]:
# Se renombran las columnas

SPCompanies.columns=['sym','name','sector','sub_sector','loc','date_added','cik_id','date_founded']

# Se ordenan las columnas 

SPCompanies = SPCompanies.iloc[:,[6,0,1,4,2,3,5,7]]

In [6]:
SPCompanies.head(3)

,cik_id,sym,name,loc,sector,sub_sector,date_added,date_founded
0,66740,MMM,3M,"Saint Paul, Minnesota",Industrials,Industrial Conglomerates,1957-03-04,1902
1,91142,AOS,A. O. Smith,"Milwaukee, Wisconsin",Industrials,Building Products,2017-07-26,1916
2,1800,ABT,Abbott,"North Chicago, Illinois",Health Care,Health Care Equipment,1957-03-04,1888


In [186]:
# Se valida valores duplicados

print (f'valores duplicados: \n{SPCompanies.duplicated().sum()}')

# se valida valores nulos

print (f'\nvalores nulos: \n{SPCompanies.isnull().sum()}')

valores duplicados: 
0

valores nulos: 
cik_id           0
sym              0
name             0
loc              0
sector           0
sub_sector       0
date_added      10
date_founded     0
dtype: int64


In [4]:
# Se localiza los registros con valores nulos

filas_con_nulos = SPCompanies[SPCompanies.isnull().any(axis=1)]
filas_con_nulos

,cik_id,sym,name,loc,sector,sub_sector,date_added,date_founded
155,715957,D,Dominion Energy,"Richmond, Virginia",Utilities,Electric Utilities,NaN,1983
212,831259,FCX,Freeport-McMoRan,"Phoenix, Arizona",Materials,Copper,NaN,1912
244,49071,HUM,Humana,"Louisville, Kentucky",Health Care,Managed Health Care,NaN,1961
405,1024478,ROK,Rockwell Automation,"Milwaukee, Wisconsin",Industrials,Electrical Components & Equipment,NaN,1903
436,1113169,TROW,T. Rowe Price,"Baltimore, Maryland",Financials,Asset Management & Custody Banks,NaN,1937
446,97476,TXN,Texas Instruments,"Dallas, Texas",Information Technology,Semiconductors,NaN,1930
458,36104,USB,U.S. Bank,"Minneapolis, Minnesota",Financials,Diversified Banks,NaN,1968
482,823768,WM,Waste Management,"Houston, Texas",Industrials,Environmental & Facilities Services,NaN,1968
490,106535,WY,Weyerhaeuser,"Seattle, Washington",Real Estate,Specialized REITs,NaN,1900
491,106640,WHR,Whirlpool Corporation,"Benton Harbor, Michigan",Consumer Discretionary,Household Appliances,NaN,1911


- Se investiga el año de ingreso al S&P 500 de las compañias con datos nulos. 
- Se genera una nueva columna 'year_in', esta establecera los años que ha estado una compañia dentro de S&P 500.  

In [13]:
# Se imputan los datos de ingreso al S&P 500 [source: "https://www.spglobal.com/spdji/en/indices/equity/sp-500/"]

SPCompanies.loc[155,'date_added']='2016'
SPCompanies.loc[212,'date_added']='2007'
SPCompanies.loc[244,'date_added']='1979'
SPCompanies.loc[405,'date_added']='2002'
SPCompanies.loc[436,'date_added']='2006'
SPCompanies.loc[446,'date_added']='1953'
SPCompanies.loc[458,'date_added']='1998'
SPCompanies.loc[482,'date_added']='1999'
SPCompanies.loc[490,'date_added']='1957'
SPCompanies.loc[491,'date_added']='1954'


# Se selecciona solo el año yyyy de las columna 'date_added' que esta en formato yyyy/mm/dd

SPCompanies['added'] = SPCompanies['date_added'].str.slice(0, 4).astype(int)

# Se obtienen los años que llevan las compañias en S&P 500 hasta la fecha

SPCompanies['years_in']= 2023-SPCompanies['added']


''' Se ignoran SettingWithCopyWarning: '''


# Se eliminan columnas 

SPCompanies=SPCompanies.drop(['date_added', 'date_founded','added'], axis=1) 

In [7]:
# Se crea función para validar caracteres especiales 

def caracteres_especiales(df_columna):
    especiales = df_columna.str.contains('[.^@_,/*?]')
    print(f'la columna {df_columna.name} contiene los siguientes caracteres especiales:')
    return df_columna.loc[especiales]

# Se busca en la columna 'sym'

caracteres_especiales (SPCompanies['sym'])

la columna sym contiene los siguientes caracteres especiales:


64    BRK.B
80     BF.B
Name: sym, dtype: object

In [18]:
# Se reemplaza el "." en la columna 'sym'

SPCompanies['sym'] = SPCompanies['sym'].str.replace(".", "-", regex=True)


SPCompanies.head(5)

,cik_id,sym,name,loc,sector,sub_sector,years_in
0,66740,MMM,3M,"Saint Paul, Minnesota",Industrials,Industrial Conglomerates,66
1,91142,AOS,A. O. Smith,"Milwaukee, Wisconsin",Industrials,Building Products,6
2,1800,ABT,Abbott,"North Chicago, Illinois",Health Care,Health Care Equipment,66
3,1551152,ABBV,AbbVie,"North Chicago, Illinois",Health Care,Pharmaceuticals,11
4,1467373,ACN,Accenture,"Dublin, Ireland",Information Technology,IT Consulting & Other Services,12


## Dataset 2 Historial valor de acciones compañias S&P 500

Se emplea Yahoo Finance para obtener la información economica de las empresas presentes en el S&P 500 durante los ultimos 23 años.

In [19]:
# Se crea una lista de los tickets presentes en el dataframe SPCompanies. 

companies_syms = SPCompanies['sym'].values.tolist()
companies_syms

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ATVI',
 'ADM',
 'ADBE',
 'ADP',
 'AAP',
 'AES',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AMD',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BALL',
 'BAC',
 'BBWI',
 'BAX',
 'BDX',
 'WRB',
 'BRK-B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF-B',
 'BG',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CDAY',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA'

In [22]:
# Se descaga los datos del precio de cierre ajustado 'Adj Close' de las acciones de las compañias

px= yf.download(companies_syms, start='2000-01-01', end='2023-02-01', rounding=True)['Adj Close']

[*********************100%***********************]  503 of 503 completed


In [23]:
# Se genera el dataframe

px1 = px.reset_index()
px1.head(5)


,Date,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
0,2000-01-03,44.00,NaN,NaN,0.85,NaN,2.99,9.22,1.28,NaN,...,NaN,7.24,18.98,6.77,NaN,4.77,NaN,25.03,37.66,NaN
1,2000-01-04,40.64,NaN,NaN,0.78,NaN,2.78,8.96,1.27,NaN,...,NaN,7.40,18.62,6.77,NaN,4.67,NaN,24.67,35.84,NaN
2,2000-01-05,38.12,NaN,NaN,0.79,NaN,3.00,8.94,1.39,NaN,...,NaN,7.69,19.64,6.89,NaN,4.70,NaN,25.14,35.79,NaN
3,2000-01-06,36.67,NaN,NaN,0.72,NaN,3.23,9.25,1.38,NaN,...,NaN,7.62,20.65,6.90,NaN,4.66,NaN,23.78,36.30,NaN
4,2000-01-07,39.72,NaN,NaN,0.76,NaN,3.65,9.35,1.45,NaN,...,NaN,7.62,20.59,6.88,NaN,4.55,NaN,23.51,36.39,NaN


In [24]:

import datetime as dt

# Se agrupa por año los promedios de el precio en que cerró las acciones de las compañias

px1=px1.groupby(px1['Date'].dt.year)[px1.columns].mean()


px1.head(3)

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2000,42.612698,NaN,NaN,0.694762,NaN,5.782063,11.124127,1.685516,NaN,29.310635,...,NaN,8.977857,20.315238,9.071468,NaN,3.891944,NaN,22.327103,33.280278,NaN
2001,20.429597,NaN,12.997727,0.307379,NaN,10.922218,13.576694,1.969637,13.048468,18.462621,...,NaN,11.657056,20.671976,12.236935,NaN,5.445242,25.814206,20.321048,37.724395,NaN
2002,14.023929,NaN,15.163452,0.290992,NaN,13.104444,12.654167,3.103413,14.747024,14.862063,...,7.071957,7.441865,19.316230,16.306389,NaN,7.148532,31.689405,24.501230,35.043690,NaN


In [25]:
# Se usa la transpuesta para una mejor visualización
px1=px1.T

# Resetea Index
px1=px1.reset_index()

# Se modifica nombre a columna de las compañias
px1 = px1.rename(columns={'index': 'sym'})
px1

Date,sym,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,A,42.612698,20.429597,14.023929,12.370000,16.427063,16.537262,20.861753,23.282390,19.556996,...,37.719325,37.135794,41.121071,56.837171,64.902151,73.335119,91.020593,143.263849,132.414462,154.0140
1,AAL,NaN,NaN,NaN,NaN,NaN,27.404478,41.784701,33.629442,7.298814,...,36.755794,42.940198,36.716349,46.101195,41.898247,30.571825,15.271621,20.343214,15.269920,15.6710
2,AAP,NaN,12.997727,15.163452,18.373810,23.902619,34.152421,33.151275,33.817251,32.389170,...,122.345278,153.151746,145.735635,115.060159,131.012669,149.388968,133.022095,191.769286,186.221992,150.0255
3,AAPL,0.694762,0.307379,0.290992,0.281944,0.539762,1.418532,2.152590,3.898964,4.315652,...,20.561270,27.208095,24.201032,35.488327,45.243108,50.625079,93.773320,139.587579,154.146494,135.5710
4,ABBV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.721032,44.695397,45.319643,58.355259,78.153386,65.653651,81.035929,105.845714,145.062829,153.0580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,3.891944,5.445242,7.148532,7.265516,10.045516,12.807222,13.562430,17.532390,18.665020,...,45.303373,50.010000,52.895119,65.430398,77.742231,97.441270,88.743913,116.094444,117.447251,128.1310
499,ZBH,NaN,25.814206,31.689405,44.838016,69.312460,67.371230,59.608247,71.492271,57.642411,...,91.949286,99.426548,104.779683,110.212869,110.084781,121.348571,127.133320,146.745794,115.600558,125.0985
500,ZBRA,22.327103,20.321048,24.501230,33.005595,52.081627,44.097341,37.826733,37.544821,29.552688,...,71.196667,89.878492,64.032024,99.310319,150.661315,206.708492,266.642490,518.283373,343.300319,293.8120
501,ZION,33.280278,37.724395,35.043690,37.528333,44.998690,53.290357,61.558805,57.849044,30.728538,...,24.547381,24.094008,24.514167,38.088566,45.483944,41.532024,32.804190,54.069286,56.120279,50.7540


Se realiza una concatenación de los dos dataframes para determinar que compañia ha mostrado los mejores rendimientos en el S&P 500

In [26]:
# Join de los dataframes
df=pd.merge(SPCompanies[['sym','name','years_in','sector']],px1, on='sym', how='inner')

df.head(9)

,sym,name,years_in,sector,2000,2001,2002,2003,2004,2005,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,MMM,3M,66,Industrials,25.200357,31.196935,34.838571,40.182976,48.675238,47.161429,...,109.134167,122.420238,135.597024,169.364542,178.208207,156.187976,143.989012,174.957500,134.120837,120.1905
1,AOS,A. O. Smith,6,Industrials,1.965556,2.016935,3.157500,3.656746,3.558770,3.719683,...,21.640913,30.359563,38.260397,49.987331,53.185378,45.092579,45.607708,66.996429,60.582351,60.3715
2,ABT,Abbott,66,Health Care,11.124127,13.576694,12.654167,11.671706,12.824444,14.371508,...,34.765238,40.130635,35.929087,44.160677,59.562749,75.982063,92.162806,118.276548,110.497968,111.6425
3,ABBV,AbbVie,11,Health Care,NaN,NaN,NaN,NaN,NaN,NaN,...,38.721032,44.695397,45.319643,58.355259,78.153386,65.653651,81.035929,105.845714,145.062829,153.0580
4,ACN,Accenture,12,Information Technology,NaN,13.048468,14.747024,14.160714,18.274484,18.302857,...,69.978016,85.882976,101.475238,118.477769,149.153267,172.536349,205.397628,302.624008,297.484582,275.4665
5,ATVI,Activision Blizzard,8,Communication Services,0.857778,2.246452,2.887976,2.012976,3.677183,5.999722,...,19.499286,26.370000,36.482619,54.497570,67.455020,48.220952,72.100198,84.391349,76.932470,76.0045
6,ADM,ADM,66,Consumer Staples,6.003135,8.261169,8.298968,8.186310,11.348492,15.076349,...,36.264643,36.826905,33.586349,36.383865,40.115299,37.723413,40.266364,58.689365,83.816892,85.1715
7,ADBE,Adobe Inc.,26,Information Technology,29.310635,18.462621,14.862063,17.505476,23.086548,30.765476,...,67.453056,80.967817,97.325040,143.996135,235.037450,279.322817,415.971265,560.613651,396.121514,349.5705
8,ADP,ADP,42,Industrials,27.215040,26.555766,22.912103,18.006984,21.813056,22.726111,...,59.840833,71.294683,77.144802,94.134701,119.171793,147.685675,143.763755,193.669444,225.213825,234.1350


Para determinar que día es idoneo para invertir se realiza una agrupación por días de la semana al dataset original.<br>
Donde 0=lunes y 6=domingo

In [40]:
# Se genera el dataframe px2

px2 = px.reset_index()
px2.head(5)


,Date,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
0,2000-01-03,44.00,NaN,NaN,0.85,NaN,2.99,9.22,1.28,NaN,...,NaN,7.24,18.98,6.77,NaN,4.77,NaN,25.03,37.66,NaN
1,2000-01-04,40.64,NaN,NaN,0.78,NaN,2.78,8.96,1.27,NaN,...,NaN,7.40,18.62,6.77,NaN,4.67,NaN,24.67,35.84,NaN
2,2000-01-05,38.12,NaN,NaN,0.79,NaN,3.00,8.94,1.39,NaN,...,NaN,7.69,19.64,6.89,NaN,4.70,NaN,25.14,35.79,NaN
3,2000-01-06,36.67,NaN,NaN,0.72,NaN,3.23,9.25,1.38,NaN,...,NaN,7.62,20.65,6.90,NaN,4.66,NaN,23.78,36.30,NaN
4,2000-01-07,39.72,NaN,NaN,0.76,NaN,3.65,9.35,1.45,NaN,...,NaN,7.62,20.59,6.88,NaN,4.55,NaN,23.51,36.39,NaN


In [41]:
# Se agrupa por día los promedios de el precio en que cerró las acciones de las compañias

px2=px2.groupby(px2['Date'].dt.dayofweek)[px2.columns].mean()


# Se usa la transpuesta para una mejor visualización
px2=px2.T

# Resetea Index
px2=px2.reset_index()

# Se modifica nombre a columna de las compañias
px2 = px2.rename(columns={'index': 'sym'})

# Join de los dataframes
df_day=pd.merge(SPCompanies[['sym','name','sector']],px2, on='sym', how='inner')

df_day.head(9)

,sym,name,sector,0,1,2,3,4
0,MMM,3M,Industrials,87.447815,87.711394,87.514279,87.637020,87.559639
1,AOS,A. O. Smith,Industrials,21.533563,21.722292,21.596988,21.641366,21.612569
2,ABT,Abbott,Health Care,37.045023,37.258749,37.109513,37.221452,37.127603
3,ABBV,AbbVie,Health Care,69.386442,69.998203,69.931865,69.967578,70.018031
4,ACN,Accenture,Information Technology,87.398437,88.368002,87.877806,88.104606,87.828932
5,ATVI,Activision Blizzard,Communication Services,25.552681,25.812536,25.689346,25.840436,25.682448
6,ADM,ADM,Consumer Staples,28.139541,28.368950,28.272500,28.281640,28.275756
7,ADBE,Adobe Inc.,Information Technology,117.558264,118.561411,117.854941,118.251230,117.642354
8,ADP,ADP,Industrials,66.427668,67.014442,66.627290,66.731076,66.754201


Se exportan los dataframes para su posterior análisis.

In [43]:
#df_day.to_csv("../datasets/df_day.csv",index=False)

In [113]:
#df.to_csv("../datasets/df.csv",index=False)